# Sensor Based Activity Recoginition 
Challenge: cdl1 - Sensor based Activity Recognition  
Team: Lea Bütler, Manjavy Kirupa, Etienne Roulet, Si Ben Tran  

Aufgabe: DL Modell erstellen

Hier in diesem Notebook erstellen wir unsere Deep Learning Modelle.

In [18]:
import tensorflow as tf
from sklearn.model_selection import KFold
import numpy as np
import logging
from datetime import datetime
import dataclasses
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn import model_selection as ms
from sklearn.model_selection import train_test_split, TimeSeriesSplit
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
import os
from tensorflow.keras.callbacks import TensorBoard
import matplotlib.pyplot as plt
import numpy as np

tf.debugging.set_log_device_placement(False)

# datetime as filename for logging
now = datetime.now()
date_time_string = now.strftime("%Y-%m-%d_%H-%M-%S")

logging.basicConfig(
    level=logging.INFO, filename=f"{date_time_string}.txt", filemode="a"
)
print(tf.config.list_physical_devices())

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


# Daten einlesen

In [19]:
from dataclasses import dataclass


# Static Parameters
@dataclass
class Parameters:
    batch_size: int = 128
    epochs: int = 15
    verbosity: str = "auto"
    step_size: int = 374
    number_folds: int = 4
    output_size: int = 6

In [36]:
df = pd.read_csv("../Sensor_Data-Wrangling-und-EDA/Alle_Messungen_trimmed.csv")
df.head()

,Unnamed: 0,time,Accelerometer_x,Accelerometer_y,Accelerometer_z,Gyroscope_x,Gyroscope_y,Gyroscope_z,Magnetometer_x,Magnetometer_y,Magnetometer_z,Orientation_qx,Orientation_qy,Orientation_qz,id,user,class,id_combined
0,1,2023-02-27 15:02:17.156,0.000,9.722,1.278,0.0,0.0,0.0,0.000000,0.0,0.000000,-0.21775,-0.719579,0.631111,01_SamsungA22-2023-02-27_15-02-03,Ben_Tran,Laufen,01_SamsungA22-2023-02-27_15-02-03Ben_TranLaufen
1,2,2023-02-27 15:02:31.574,-0.728,9.779,0.000,0.0,0.0,0.0,0.000000,0.0,0.000000,-0.42065,0.568356,-0.598058,01_SamsungA22-2023-02-27_15-02-03,Ben_Tran,Laufen,01_SamsungA22-2023-02-27_15-02-03Ben_TranLaufen
2,3,2023-02-27 15:02:16.685,0.000,0.000,0.000,0.0,0.0,0.0,0.000000,43.5,-13.143750,0.00000,0.000000,0.000000,01_SamsungA22-2023-02-27_15-02-03,Ben_Tran,Laufen,01_SamsungA22-2023-02-27_15-02-03Ben_TranLaufen
3,4,2023-02-27 15:02:32.295,0.000,0.000,0.000,0.0,0.0,0.0,-13.012501,0.0,20.587502,0.00000,0.000000,0.000000,01_SamsungA22-2023-02-27_15-02-03,Ben_Tran,Laufen,01_SamsungA22-2023-02-27_15-02-03Ben_TranLaufen
4,5,2023-02-27 15:02:33.285,0.000,0.000,0.000,0.0,0.0,0.0,-22.612501,0.0,20.268751,0.00000,0.000000,0.000000,01_SamsungA22-2023-02-27_15-02-03,Ben_Tran,Laufen,01_SamsungA22-2023-02-27_15-02-03Ben_TranLaufen


## Time umwandeln 

In [37]:
# convert the string time column to datetime
epoch = pd.Timestamp("1970-01-01")

df["time"] = pd.to_datetime(df["time"])
df["time"] = (df["time"] - epoch).apply(lambda x: int(x.total_seconds() * 1000))

print(df.shape)

(2344488, 18)


## Create Validation Data

In [38]:
# select three random id_combines files
ids = ["06_iPhone12-2023-03-16_13-46-58Manjavy_KirupaVelofahren", "01_iPhone13pro-2023-03-21_16-55-47Etienne_RouletLaufen", "01_iPhone13ProMax-2023-03-15_18-29-42Gabriel_TorresRennen"]
df_validation = df[df["id_combined"].isin(ids)]

# export to csv 
df_validation.to_csv("validation-velo-laufen-rennen.csv", index=False)

# remove the validation data from the dataframe 
df = df[~df["id_combined"].isin(ids)]

## Drop unnötige Spalten und Class Encoden

In [34]:
df = df.drop(columns=["id", "user", "id_combined"])
df.drop(["Unnamed: 0"], axis=1, inplace=True)
# get all types of the df
le = LabelEncoder()
df["class"] = le.fit_transform(df["class"])
# print dictionary of the classes and its encoded values
print(dict(zip(le.classes_, le.transform(le.classes_))))

{'Laufen': 0, 'Rennen': 1, 'Sitzen': 2, 'Stehen': 3, 'Treppenlaufen': 4, 'Velofahren': 5}


## Daten Transformation

In [7]:
# Set the window size and step size
window_size = 300
step_size = 100

# Reshape X to 2D format (samples, features)
X = df.values[:, 1:13]

# Define y
y = df["class"].values

# Create a sliding window of X with the specified window and step sizes
X_windows = np.array(
    [
        X[i : i + window_size, :]
        for i in range(0, X.shape[0] - window_size + 1, step_size)
    ]
)

# Reshape X_windows to 3D format (samples, timesteps, features)
timesteps = X_windows.shape[1]
n_features = X_windows.shape[2]
X_windows = X_windows.reshape(-1, timesteps, n_features)

# Create the corresponding y labels for the sliding windows
y_windows = np.array(
    [y[i + window_size - 1] for i in range(0, X.shape[0] - window_size + 1, step_size)]
)
y_windows = to_categorical(y_windows, num_classes=6)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X_windows, y_windows, test_size=0.2, random_state=42, stratify=y_windows
)

In [8]:
X.shape, y.shape, X_train.shape, y_train.shape, X_test.shape, y_test.shape

((2340425, 12),
 (2340425,),
 (18721, 300, 12),
 (18721, 6),
 (4681, 300, 12),
 (4681, 6))

## Umschreibung

In [9]:
x_train = X_train
x_test = X_test
y_train = y_train
y_test = y_test

# DL-Modelle erstellen

In [10]:
# Something like this as first Model

def create_model_1(name="model_1"):
    model = tf.keras.Sequential(
        [
            # Add a 1D convolutional layer
            tf.keras.layers.Conv1D(
                filters=64,
                kernel_size=12,
                activation="relu",
                padding="same",
                input_shape=(timesteps, n_features),
                kernel_regularizer=tf.keras.regularizers.l2(0.01),
            ),
            # Add LSTM layer
            tf.keras.layers.LSTM(100),
            # Add a dense output layer
            tf.keras.layers.Dense(
                6, activation="softmax"
            ),  # Change activation function based on the nature of the output
        ],
        name=name,
    )
    model.compile(
        optimizer="adam",
        loss="categorical_crossentropy",
        metrics=["accuracy", tf.keras.metrics.Precision(), tf.keras.metrics.Recall()],
    )
    return model


def create_model_2(name="model_2"):
    model = tf.keras.Sequential(
        [
            # Add a 1D convolutional layer
            tf.keras.layers.Conv1D(
                filters=32,
                kernel_size=4,
                activation="relu",
                padding="same",
                input_shape=(timesteps, n_features),
                kernel_regularizer=tf.keras.regularizers.l2(0.01),
            ),
            # Add LSTM layer
            tf.keras.layers.LSTM(100),
            # Add a dense output layer
            tf.keras.layers.Dense(
                6, activation="softmax"
            ),  # Change activation function based on the nature of the output
        ],
        name=name,
    )
    model.compile(
        optimizer="adam",
        loss="categorical_crossentropy",
        metrics=["accuracy", tf.keras.metrics.Precision(), tf.keras.metrics.Recall()],
    )
    return model

def create_model_3(name="model_3"):
    model = tf.keras.Sequential(
        [
            # Add a 1D convolutional layer
            tf.keras.layers.Conv1D(
                filters=32,
                kernel_size=2,
                activation="relu",
                padding="same",
                input_shape=(timesteps, n_features),
                kernel_regularizer=tf.keras.regularizers.l2(0.01),
            ),
            # Add LSTM layer
            tf.keras.layers.LSTM(100),
            # Add a dense output layer
            tf.keras.layers.Dense(
                6, activation="softmax"
            ),  # Change activation function based on the nature of the output
        ],
        name=name,
    )
    model.compile(
        optimizer="adam",
        loss="categorical_crossentropy",
        metrics=["accuracy", tf.keras.metrics.Precision(), tf.keras.metrics.Recall()],
    )
    return model


def create_model_4(name="model_4"):
    model = tf.keras.Sequential(
        [
            # Add a 1D convolutional layer
            tf.keras.layers.Conv1D(
                filters=64,
                kernel_size=2,
                activation="relu",
                padding="same",
                input_shape=(timesteps, n_features),
                kernel_regularizer=tf.keras.regularizers.l2(0.01),
            ),
            # Add LSTM layer
            tf.keras.layers.LSTM(100),
            # Add a dense output layer
            tf.keras.layers.Dense(
                6, activation="softmax"
            ),  # Change activation function based on the nature of the output
        ],
        name=name,
    )
    model.compile(
        optimizer="adam",
        loss="categorical_crossentropy",
        metrics=["accuracy", tf.keras.metrics.Precision(), tf.keras.metrics.Recall()],
    )
    return model


def create_model_5(name="model_5"):
    model = tf.keras.Sequential(
        [
            # Add a 1D convolutional layer
            tf.keras.layers.Conv1D(
                filters=32,
                kernel_size=2,
                activation="relu",
                padding="same",
                input_shape=(timesteps, n_features),
                kernel_regularizer=tf.keras.regularizers.l2(0.01),
            ),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.MaxPooling1D(pool_size=2),
            tf.keras.layers.Conv1D(
                filters=20, kernel_size=2, activation="relu", padding="same",
                kernel_regularizer=tf.keras.regularizers.l2(0.01),
            ),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.MaxPooling1D(pool_size=2),
            tf.keras.layers.Conv1D(
                filters=10, kernel_size=2, activation="relu", padding="same", kernel_regularizer=tf.keras.regularizers.l2(0.01),
            ),
            tf.keras.layers.MaxPooling1D(pool_size=2),
            # flatten output
            tf.keras.layers.Flatten(),
            # Add a dense output layer
            tf.keras.layers.Dense(180, activation="relu", kernel_regularizer=tf.keras.regularizers.l2(0.01),),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Dense(100, activation="relu", kernel_regularizer=tf.keras.regularizers.l2(0.01),),
            tf.keras.layers.BatchNormalization(),
            # Change activation function based on the nature of the output
            tf.keras.layers.Dense(6, activation="softmax"),
        ],
        name=name,
    )

    model.compile(
        optimizer="adam",
        loss="categorical_crossentropy",
        metrics=["accuracy", tf.keras.metrics.Precision(), tf.keras.metrics.Recall()],
    )
    return model

# DL-Modelle Trainieren

In [11]:
best_model_history = None  # Keep track of the best model's history
model_histories = []
# Perform cross-validation
models = [create_model_1,create_model_2,create_model_3,create_model_4, create_model_5]
best_model = None
num_folds = Parameters.number_folds
tscv = TimeSeriesSplit(n_splits=num_folds)
fold_acc_scores = []


for i, (train, test) in enumerate(tscv.split(x_train)):
    logging.info(f"Fold {i+1}")
    train_x, train_y = x_train[train], y_train[train]
    test_x, test_y = x_train[test], y_train[test]

    fold_histories = []

    for j, model_creator in enumerate(models):
        print(f"Model {model_creator.__name__}")
        model_name = f"Model_{j+1}_Fold_{i+1}"
        model = model_creator(name=model_name)
        logging.info(f"Model {j+1}")
        history = model.fit(
            train_x,
            train_y,
            epochs=Parameters.epochs,
            batch_size=Parameters.batch_size,
            validation_data=(test_x, test_y),
            verbose=Parameters.verbosity,
        )
        test_loss, acc, prec, recal = model.evaluate(
            test_x, test_y, verbose=Parameters.verbosity
        )
        logging.info(f"Validation accuracy: {acc}")

        fold_histories.append(history.history)

        for epoch in range(Parameters.epochs):
            # Log accuracy after each epoch
            acc_epoch = history.history["val_accuracy"][epoch]
            logging.info(f"Epoch {epoch + 1}, Validation accuracy: {acc_epoch}")
        fold_acc_scores.append((i, j, acc))

        if best_model_history is None or acc > best_model_acc:
            best_model_history = history
            best_model = model  # Store the trained model instance
            best_model_acc = acc

    model_histories.append(fold_histories)

# Find the best model
best_model_index = np.argmax([score[2] for score in fold_acc_scores])
best_fold_idx, best_model_idx, _ = max(fold_acc_scores, key=lambda x: x[2])
best_model_history = model_histories[best_fold_idx][best_model_idx]
print(best_model.name)

Model create_model_1
Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB

Epoch 1/15


2023-05-08 16:24:49.941693: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


30/30 [==============================] - 4s 105ms/step - loss: 1.5144 - accuracy: 0.4943 - precision: 0.7517 - recall: 0.2013 - val_loss: 1.2495 - val_accuracy: 0.5908 - val_precision: 0.7534 - val_recall: 0.3990
Epoch 2/15
30/30 [==============================] - 2s 81ms/step - loss: 1.1094 - accuracy: 0.6619 - precision: 0.8282 - recall: 0.4299 - val_loss: 0.9924 - val_accuracy: 0.7081 - val_precision: 0.8215 - val_recall: 0.5200
Epoch 3/15
30/30 [==============================] - 2s 80ms/step - loss: 0.8543 - accuracy: 0.7629 - precision: 0.8673 - recall: 0.6163 - val_loss: 0.7506 - val_accuracy: 0.7994 - val_precision: 0.8755 - val_recall: 0.7083
Epoch 4/15
30/30 [==============================] - 2s 82ms/step - loss: 0.6845 - accuracy: 0.8152 - precision: 0.8832 - recall: 0.7348 - val_loss: 0.6768 - val_accuracy: 0.8157 - val_precision: 0.8799 - val_recall: 0.7439
Epoch 5/15
30/30 [==============================] - 2s 80ms/step - loss: 0.5790 - accuracy: 0.8563 - precision: 0.9021

## Loss und Accuracy Verlauf vom besten Modell

In [ ]:
print(best_model.name)
print(best_model_history["loss"])
# summarize history for accuracy
plt.plot(best_model_history["accuracy"])
plt.plot(best_model_history["val_accuracy"])
plt.title("model accuracy")
plt.ylabel("accuracy")
plt.xlabel("epoch")
plt.legend(["train", "test"], loc="upper left")
plt.show()
# summarize history for loss
plt.plot(best_model_history["loss"])
plt.plot(best_model_history["val_loss"])
plt.title("model loss")
plt.ylabel("loss")
plt.xlabel("epoch")
plt.legend(["train", "test"], loc="upper left")
plt.show()

## Confusions Matrix

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

# for multiclass classification
y_pred = best_model.predict(x_test)
y_test_labels = y_test.argmax(axis=1)
y_pred_labels = y_pred.argmax(axis=1)

# create cm
cm = confusion_matrix(y_test_labels, y_pred_labels)
display(le.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=le.classes_)
# get f1 score of each class
from sklearn.metrics import f1_score

f1 = f1_score(y_test_labels, y_pred_labels, average=None)
# get f1 score of each class

# plot confusion matrix
disp.plot()
plt.show()
display(f1)

## Validation Daten Predicten

In [ ]:
df_val = pd.read_csv("validation_data.csv")
df_val = df_val.drop(columns=["id", "user", "id_combined"])
# convert the string time column to datetime
epoch = pd.Timestamp("1970-01-01")
df_val["time"] = pd.to_datetime(df_val["time"])
df_val["time"] = (df_val["time"] - epoch).apply(
    lambda x: int(x.total_seconds() * 1000)
)
df_val.drop(["Unnamed: 0"], axis=1, inplace=True)
# get all types of the df
le = LabelEncoder()
df_val["class"] = le.fit_transform(df_val["class"])

# Set the window size and step size
window_size = 300
step_size = 100

# Reshape X to 2D format (samples, features)
X = df_val.values[:, 1:13]

# Create a sliding window of X with the specified window and step sizes
X_windows = np.array(
    [
        X[i : i + window_size, :]
        for i in range(0, X.shape[0] - window_size + 1, step_size)
    ]
)

# Reshape X_windows to 3D format (samples, timesteps, features)
timesteps = X_windows.shape[1]
n_features = X_windows.shape[2]
X_windows = X_windows.reshape(-1, timesteps, n_features)

y_pred_probs = best_model.predict(X_windows)

# Get the predicted class labels for each input window
y_pred_labels = np.argmax(y_pred_probs, axis=1)

# Print the predicted class labels
# get the median of the predicted labels
sol = np.median(y_pred_labels)

class_counts = np.bincount(y_pred_labels)
for i, count in enumerate(class_counts):
    print(f"Class {i} count: {count}")

# Export bestes DL-Modell

In [ ]:
# export Model to json Tensorflow file
import json

best_model.save("saved_model/sensor_model.h5")

model = tf.keras.models.load_model("saved_model/sensor_model.h5")

# Save model architecture to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

# Save model weights to a JSON file
weights = model.get_weights()
weights_as_list = [w.tolist() for w in weights]
with open("weights.json", "w") as f:
    json.dump(weights_as_list, f)

# Modell Einlesen und predicten

In [30]:
# Model laden
model = tf.keras.models.load_model("saved_model/sensor_model.h5")

df_val = pd.read_csv("validation_data.csv")
df_val = df_val.drop(columns=["id", "user", "id_combined"])
# convert the string time column to datetime
epoch = pd.Timestamp("1970-01-01")
df_val["time"] = pd.to_datetime(df_val["time"])
df_val["time"] = (df_val["time"] - epoch).apply(
    lambda x: int(x.total_seconds() * 1000)
)
df_val.drop(["Unnamed: 0"], axis=1, inplace=True)
# get all types of the df
le = LabelEncoder()
df_val["class"] = le.fit_transform(df_val["class"])

# Set the window size and step size
window_size = 300
step_size = 100

# Reshape X to 2D format (samples, features)
X = df_val.values[:, 1:13]

# Create a sliding window of X with the specified window and step sizes
X_windows = np.array(
    [
        X[i : i + window_size, :]
        for i in range(0, X.shape[0] - window_size + 1, step_size)
    ]
)

# Reshape X_windows to 3D format (samples, timesteps, features)
timesteps = X_windows.shape[1]
n_features = X_windows.shape[2]
X_windows = X_windows.reshape(-1, timesteps, n_features)

y_pred_probs = model.predict(X_windows)

# Get the predicted class labels for each input window
y_pred_labels = np.argmax(y_pred_probs, axis=1)

# Print the predicted class labels
# get the median of the predicted labels
sol = np.median(y_pred_labels)

class_counts = np.bincount(y_pred_labels)
for i, count in enumerate(class_counts):
    print(f"Class {i} count: {count}")

8/8 [==============================] - 5s 140ms/step
Class 0 count: 41
Class 1 count: 0
Class 2 count: 1
Class 3 count: 1
Class 4 count: 28
Class 5 count: 171
